In [1]:
import os

In [2]:
# switch to the path where the dataset is stored
%cd /content/drive/MyDrive/dataset_images/

/content/drive/MyDrive/dataset_images


Don't forget to modify my_dataset.py when you want to change the dataset

In [3]:
# copy the zip file (dataset) to the /content directory

!cp SSMFVD.zip /content
#!cp kaggle.zip /content
#!cp PWMFD.zip /content

In [ ]:
# # switch to content directory
%cd /content
# unzip
!unzip SSMFVD.zip -d '/content/dataset'
#!unzip kaggle.zip -d '/content/dataset'
#!unzip PWMFD.zip -d '/content/dataset'

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/train/26_1_11.txt  
  inflating: /content/dataset/train/23_1_4.xml  
  inflating: /content/dataset/train/21_10_2.txt  
  inflating: /content/dataset/train/21_2_25.txt  
  inflating: /content/dataset/train/14_4_24.xml  
  inflating: /content/dataset/train/20_3_13.jpg  
  inflating: /content/dataset/train/27_1_11.txt  
  inflating: /content/dataset/train/20_2_31.txt  
  inflating: /content/dataset/train/17_2_1.jpg  
  inflating: /content/dataset/train/6_2_18.txt  
  inflating: /content/dataset/train/3_2_12.txt  
  inflating: /content/dataset/train/28_2_17.xml  
  inflating: /content/dataset/train/4_1_32.txt  
  inflating: /content/dataset/train/16_2_4.xml  
  inflating: /content/dataset/train/25_3_19.jpg  
  inflating: /content/dataset/train/25_4_10.jpg  
  inflating: /content/dataset/train/22_1_1.jpg  
  inflating: /content/dataset/train/22_6_12.txt  
  inflating: /content/dataset/train/29_2_17.xml  
  infl

In [ ]:
# switch to the path where the dataset txt files are stored

%cd /content/drive/MyDrive/TrainingStage2/data/own_video_dataset_txt/

#%cd /content/drive/MyDrive/TrainingStage2/data/kaggle_dataset_txt

#%cd /content/drive/MyDrive/TrainingStage2/data/PWMFD_dataset_txt

/content/drive/MyDrive/faster_rcnn/data/own_video_dataset_txt


In [ ]:
# copy the txt files to /content/dataset/
!cp train.txt /content/dataset/ #
!cp valid.txt /content/dataset/ #
!cp test.txt /content/dataset/ #

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import sys
print(sys.path)
sys.path.append('/content/drive/MyDrive/TrainingStage2/')
# print(sys.path)

In [ ]:
import os
import datetime

import torch

import transforms
from network_files import FasterRCNN, FastRCNNPredictor
from backbone import resnet50_fpn_backbone
from my_dataset import MyDataSet
from train_utils import train_eval_utils as utils

In [ ]:

def main(args):
    device = torch.device(args.device if torch.cuda.is_available() else "cpu")
    print("Using {} device training.".format(device.type))

    results_file = "evaluate_results{}.txt".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

    data_transform = {
        "val": transforms.Compose([transforms.ToTensor()])
    }


    ##############################################################################
    mydataset = args.data_path
    # check dataset root
    if os.path.exists(mydataset) is False:
        raise FileNotFoundError("MyDataSet dose not in path:'{}'.".format(mydataset))

    batch_size = args.batch_size
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    # load validation data set
    # /content/dataset/valid.txt
    val_dataset = MyDataSet(mydataset, "test", data_transform["val"], "test.txt")
    val_data_set_loader = torch.utils.data.DataLoader(val_dataset,
                                                      batch_size=1,
                                                      shuffle=False,
                                                      pin_memory=True,
                                                      num_workers=nw,
                                                      collate_fn=val_dataset.collate_fn)
    ##############################################################################
    backbone = resnet50_fpn_backbone(pretrain_path="/content/drive/MyDrive/TrainingStage2/backbone/resnet50.pth",
                                     norm_layer=torch.nn.BatchNorm2d, # FrozenBatchNorm2d, BatchNorm2d
                                     trainable_layers=5)
    # 90+1 or 3+1, set it as 4 when you load your own model，otherwise 91
    model = FasterRCNN(backbone=backbone, num_classes=3+1)
    model_path = args.model_path
    model_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(model_dict["model"], strict=True)
    # print(model)

    model.to(device)


    val_map_05 = []
    val_map_0595 = []

    for epoch in range(0, args.epochs):
        # evaluate on the test dataset
        coco_info = utils.evaluate(model, val_data_set_loader, device=device)

        # write into txt
        with open(results_file, "a") as f:
            result_info = [f"{i:.4f}" for i in coco_info]
            txt = "epoch:{} {}".format(epoch, '  '.join(result_info))
            f.write(txt + "\n")

        
        val_map_0595.append(coco_info[0])  # mAP @ IoU=0.50:0.95
        val_map_05.append(coco_info[1])  # mAP @ IoU=0.50

    # # plot mAP curve
    # if len(val_map_05) != 0 or len(val_map_0595) != 0:
    #     from plot_curve import plot_map
    #     plot_map(val_map_05, val_map_0595)


In [ ]:

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description=__doc__)

    parser.add_argument('--device', default='cuda:0', help='device')
    parser.add_argument('--data-path', default='/content/dataset/', help='dataset')
    parser.add_argument('--epochs', default=2, type=int, metavar='N',
                        help='number of total epochs to run')
    parser.add_argument('--batch_size', default=4, type=int, metavar='N',
                        help='batch size when training.')
    parser.add_argument('--aspect-ratio-group-factor', default=3, type=int)
    parser.add_argument("--amp", default=False, help="Use torch.cuda.amp for mixed precision training")
    parser.add_argument('--model_path', 
                        #default='/content/drive/MyDrive/TrainingStage2/save_weights-best-our-propose-result-supervised/resNetFpn-model-10.pth',
                        default='/content/drive/MyDrive/TrainingStage2/save_weights/resNetFpn-model-15.pth',
                        #default='/content/drive/MyDrive/TrainingStage2/backbone/resNetFpn-trainOnKaggle-5.pth',
                        help='evaluate model path')
    opt = parser.parse_args(args=[])
    print(opt)

    main(opt)
